# GNN for Parity Games Experiments

## Raw Training Data Creation

**07 July 2022**

- Num games: 3000
- Graph size (N) range: [10,200]
- Relative outdegree $(\frac{d_i}{N})$ range: [0.01,0.5]
- Priority range: [0,N]

In [2]:
num_graphs = 3000
min_n = 10
max_n = 200
min_rod = 0.01
max_rod = 0.5

games_dir = 'games'
solutions_dir = 'solutions'
pgsolver_base = 'pgsolver' # Path to compiled base dir of https://github.com/tcsprojects/pgsolver 

In [2]:
#import game_generator as gg

#gg.create_games_and_solutions(num_graphs, min_n, max_n, min_rod, max_rod, games_dir, solutions_dir, pgsolver_base)

## Prepare Training

In [37]:
from torch_geometric.loader import DataLoader
from parity_game_dataset import ParityGameDataset
import math

#data = ParityGameDataset('pg_data_20220708', 'games', 'solutions')
data = ParityGameDataset('pg_data_20220708', 'games_large', 'solutions_large')

# Use first 70% of the data for training
split_index = math.floor(0.7 * len(data))
#train_data = data[:split_index]
test_data = data

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

ParityGameDataset(3000)

**Training Parameters**

- optimizer: Adam
- Learning rate: 0.001
- Loss: Cross Entropy
- Epochs: 1-100

## Prepare model

**Parameters**

- Iterations: 10
- ...

In [14]:
from importlib import reload
import parity_game_network as pn
#from parity_game_network import ParityGameNetwork
reload(pn)
model = pn.ParityGameNetwork(256, 256, 10)
#model = pn.ParityGameNetwork(128, 128, 5)

In [15]:
model

ParityGameNetwork(
  (core): GCN(3, 256, num_layers=10)
  (node_classifier): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
  (edge_classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)

# Run Training Loop

In [16]:
import wandb
wandb.init(project='gnn_parity_game_solver')
config = wandb.config
config.learning_rate = 0.001

wandb.watch(model, log='all')

wandb: Currently logged in as: alexdweinert. Use `wandb login --relogin` to force relogin


[]

In [18]:
import torch
import numpy as np
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion_nodes = torch.nn.CrossEntropyLoss()
criterion_edges = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.1,0.9]))

def train():
    running_loss = 0.
    i = 0
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        i += 1
        optimizer.zero_grad()  # Clear gradients.
        out_nodes, out_edges = model(data.x, data.edge_index)  # Perform a single forward pass.
        
        # Most edges do not belong to a winning strategy and thus the data is extemely imbalanced. The model will probably learn that predicting always "non-winning" for each edge 
        # yields reasonable performance. To avoid this, approximately as many non-winning strategy edges are sampled as there are winning edges.
        #edge_selection = (torch.rand(data.y_edges.shape[0]) > 0.7) | (data.y_edges == 1)
        loss = criterion_nodes(out_nodes, data.y_nodes) + criterion_edges(out_edges, data.y_edges) # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        
        running_loss += loss.item()
        if i % 50 == 0:
            last_loss = running_loss / 50 # loss per batch
            wandb.log({'loss': last_loss, 'variance': torch.var(out_nodes[:,1])})
            running_loss = 0.
            
def test(loader):
     model.eval()

     correct_nodes = 0
     correct_edges = 0

     for data in loader:  # Iterate in batches over the training/test dataset.
        out_nodes, out_edges = model(data.x, data.edge_index)  
        pred_nodes = out_nodes.argmax(dim=1)  # Use the class with highest probability.
        pred_edges = out_edges.argmax(dim=1)  # Use the class with highest probability.
        correct_nodes += (pred_nodes == data.y_nodes).sum() / len(pred_nodes)  # Check against ground-truth labels.
        correct_edges += (pred_edges == data.y_edges).sum() / len(pred_edges)
     return (correct_nodes / len(loader), correct_edges / len(loader))  # Derive ratio of correct predictions.

for epoch in range(1, 2):
    train()
    train_acc_nodes, train_acc_edges = test(train_loader)
    test_acc_nodes, test_acc_edges = test(test_loader)
    wandb.log({'Epoch': epoch, 'Acc_Acc_Nodes': train_acc_nodes, 'Test_Acc_Nodes': test_acc_nodes, 'Train_Acc_Edges': train_acc_edges, 'Test_Acc_Edges': test_acc_edges}) 
    print(f'Epoch: {epoch:03d}, Train Acc Nodes: {train_acc_nodes:.4f}, Test Acc Nodes: {test_acc_nodes:.4f}, Train Acc Edges: {train_acc_edges:.4f}, Test Acc Edges: {test_acc_edges:.4f}')

Epoch: 001, Train Acc Nodes: 0.6019, Test Acc Nodes: 0.6004, Train Acc Edges: 0.8658, Test Acc Edges: 0.8643


*Saving the following model:*
- Performance: Epoch: 001, Train Acc Nodes: 0.9832, Test Acc Nodes: 0.9824, Train Acc Edges: 0.9687, Test Acc Edges: 0.9685

In [22]:
print(model)

ParityGameNetwork(
  (core): GCN(3, 256, num_layers=10)
  (node_classifier): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
  (edge_classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)


In [19]:
torch.save(model.state_dict(), 'GCN_pg_solver_20220914.pth')

In [8]:
import torch
import numpy as np
model.load_state_dict(torch.load('GAT_pg_solver_20220914.pth'), strict=True)

<All keys matched successfully>

## Unparsing predicted strategies

In [11]:

import pg_parser as parser
nodes, _ = parser.get_nodes_and_edges("games/game_3.txt")
game = next(iter(test_loader))
out_nodes, out_edges = model(game.x, game.edge_index)
out_edges

tensor([[6.6679e-01, 3.3321e-01],
        [6.1359e-01, 3.8641e-01],
        [9.9996e-01, 3.5264e-05],
        ...,
        [9.9967e-01, 3.3177e-04],
        [4.9355e-01, 5.0645e-01],
        [9.9982e-01, 1.7557e-04]], grad_fn=<SoftmaxBackward0>)

In [38]:

game_list = os.listdir('games_large')

game_it = iter(test_loader)
print(next(game_it))
print(game_list[0])

DataBatch(x=[68, 3], edge_index=[2, 1308], y_nodes=[68], y_edges=[1308], batch=[68], ptr=[2])
game_2803.txt


In [ ]:
import pg_parser as parser
import os
import time

game_list = os.listdir('games_large')[:100]

game_it = iter(test_loader)
with open('timings.csv', 'w') as f:
    f.write("File;Time(s)")
#for i in range(split_index, len(data)):
for game_file_name in game_list:
    
    game_file = 'games_large' + "/" + game_file_name
    nodes, edges = parser.get_nodes_and_edges(game_file)
    
    timestamp = time.time()
    out_nodes, out_edges = model(torch.tensor(nodes, dtype=torch.float), torch.tensor(edges, dtype=torch.long).t().contiguous())
    duration = time.time() - timestamp
    with open('timings.csv', 'a') as f:
        f.write(f"\n{game_file};{duration}")
    
    winning_edge_index = [None] * len(game.x)
    
    for edge_index in range(len(game.y_edges)):
            
        start_vertex = game.edge_index[0][edge_index]
        #print(f"Examining edge {edge_index} with start vertex {start_vertex} and attribute {out_edges[edge_index][0]}")
            
        if winning_edge_index[start_vertex] == None:
            winning_edge_index[start_vertex] = edge_index
            
        previous_edge_index = winning_edge_index[start_vertex]
        previous_probability = out_edges[previous_edge_index][0]
        current_probability = out_edges[edge_index][0]
        
        owner = 0 if nodes[start_vertex][1] > nodes[start_vertex][2] else 1
        
        if owner == 0 and current_probability > previous_probability:
            winning_edge_index[start_vertex] = edge_index
        elif owner == 1 and current_probability < previous_probability:
            winning_edge_index[start_vertex] = edge_index
    
    solution_path = "generated_solutions/gensolution_" + game_file_name
    with open(solution_path, 'w') as output_file:
        # We subtract one from the length to obtain the maximal vertex identifier used in the game
        output_file.write(f"paritysol {len(game.x) - 1};")
        for start_node in range(len(winning_edge_index)):
            owner = 0 if nodes[start_node][1] > nodes[start_node][2] else 1
            winner = 0 if out_nodes[start_node][0].item() > out_nodes[start_node][1].item() else 1
            
            if owner == winner:
                next_node = game.edge_index[1][winning_edge_index[start_node]]
                output_file.write(f"\n{start_node} {winner} {next_node};")
            else:
                output_file.write(f"\n{start_node} {winner};")
        output_file.write("\n") # Closing newline for strategy file
    

with open('timings.csv', 'a') as f:
    f.write(f"\n")

## Model application example

In [11]:
it = iter(test_loader)
next(it)
example_game = next(it)
out_nodes, out_edges = model(example_game.x, example_game.edge_index) 
pred_nodes = out_nodes.argmax(dim=1)
pred_edges = out_edges.argmax(dim=1)

### Output 1: Winning reagions of players 0 and 1

**Predicted regions**

In [12]:
pred_nodes

tensor([1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,

**Actual winning regions (Calculated by pgsolver)**

In [13]:
example_game.y_nodes

tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,

### Output 2: Winning strategies

A **1** means that the edge belongs to a winning strategy

**Predicted winning strategy**

In [14]:
example_game.edge_index[:,example_game.y_edges == 1]

tensor([[  0,   1,   2,  ..., 664, 665, 666],
        [ 40,   1,  26,  ..., 613, 605, 613]])

**Winning strategy from pgsolver**

In [78]:
example_game.edge_index[:,pred_edges == 1]

tensor([[  9,   9,  20, 101, 117, 117, 138, 138, 138, 138, 292, 292, 292],
        [ 12,  16,  16, 150,  51, 102,  71, 125, 153, 178, 250, 309, 334]])

In [72]:
max(pred_edges)

tensor(1)

In [43]:
example_game.y_edges.shape[0]

17852